# Testing Titanic with Decision Trees

This notebook was made for the [kaggle Titanic competition](https://www.kaggle.com/c/titanic/).

With inspiration from the [ppxgboost repository](https://github.com/awslabs/privacy-preserving-xgboost-inference/blob/master/example/Titanic.ipynb), which is "Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved. SPDX-License-Identifier: Apache-2.0".

It also took some ideas from several upvoted public notebooks, including [this one](https://www.kaggle.com/code/startupsci/titanic-data-science-solutions/notebook) from Manav Sehgal and [this one](https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy#Step-3:-Prepare-Data-for-Consumption) from LD Freeman.

## Imports

In [1]:
import os
import time

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from xgboost import XGBClassifier

from concrete.ml.sklearn import XGBClassifier as ConcreteXGBClassifier

## Preprocessing the Data

Be sure to run `make download_datasets` to have local versions of the dataset.

In [2]:
# Download the datasets
if not os.path.isfile("./local_datasets/titanic/train.csv"):
    raise ValueError("Please launch `make download_datasets` to get datasets")

train_data = pd.read_csv("./local_datasets/titanic/train.csv")
test_data = pd.read_csv("./local_datasets/titanic/test.csv")
datasets = [train_data, test_data]

# Save the passenger IDs used for submission
test_ids = test_data["PassengerId"]

Let's take a closer look at the train data:

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We can observe:
- the target variable: Survived
- some numerical variables: PassengerID, Pclass, SbSp, Parch, Fare
- some categorical (non-numerical) variables: Name, Sex, Ticket, Cabin, Embarked

First, it seems that PassengerId and Ticket are supposed to be random Ids that should not impact the predictions so we can already remove them from the data.

In [4]:
drop_column = ["PassengerId", "Ticket"]

### Missing Values

Then, we can notice that some values are missing for the Cabin variable. We must therefore investigate a bit more about this by printing the total amounts of missing values for each variables:

In [5]:
print("-" * 3, "Train data", "-" * 3)
print(train_data.isnull().sum(), "\n")
print("-" * 3, "Test data", "-" * 3)
print(test_data.isnull().sum())

--- Train data ---
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64 

--- Test data ---
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


Only four variables are incomplete: Cabin, Age, Embarked and Fare. However, Cabin seems to be missing quite a lot of them compared to the others:

In [6]:
n_cabin_missing_values = pd.concat(datasets).Cabin.isnull().sum()
n_total_values = pd.concat(datasets).shape[0]
print(f"Percentage of missing values in Cabin: {n_cabin_missing_values/n_total_values*100:.1f}%")

Percentage of missing values in Cabin: 77.5%


Since the Cabin variables misses more than 2/3 of its values, it might not be relevant to keep it:

In [7]:
drop_column += ["Cabin"]

for dataset in datasets:
    # Remove unrelevant variables
    dataset.drop(drop_column, axis=1, inplace=True)

For the other ones, we can replace the missing values using:
- the median value for Age and Fare
- the most common value for Embarked

In [8]:
for dataset in datasets:
    # Complete missing Age values with median
    dataset.Age.fillna(dataset.Age.median(), inplace=True)

    # Complete missing Embarked values with the most common one
    dataset.Embarked.fillna(dataset.Embarked.mode()[0], inplace=True)

    # Complete missing Fare values with median
    dataset.Fare.fillna(dataset.Fare.median(), inplace=True)

### Feature Engineering

We can manually extract and create new features in order to help the model interpret some behaviors and correctly predict an outcome. Those new features are:
- FamilySize: the size of the family the individual was travelling with, with 1 being someone travelling alone. 
- IsAlone: 1 if the individual was travelling alone, 0 if not. It might help the model emphasize on this idea of travelling with relatives or not.
- Title: the individual's title (Mr, Mrs, ...), which might help the model to have a better idea of each one's social status.
- Farebin and AgeBin: Binned version of the Fare and Age variables. Grouping values together might reduce the effects of minor observation errors.

In [9]:
def get_bin_labels(bin_name, number_of_bins):
    labels = []
    for i in range(number_of_bins):
        labels.append(bin_name + f"_{i}")
    return labels


for dataset in datasets:
    # Emphasize on relatives
    dataset["FamilySize"] = dataset.SibSp + dataset.Parch + 1

    dataset["IsAlone"] = 1
    dataset.IsAlone[dataset.FamilySize > 1] = 0

    # Consider an individual's social status
    dataset["Title"] = dataset.Name.str.extract(r" ([A-Za-z]+)\.", expand=False)

    # Group fares and ages in "bins" or "buckets"
    dataset["FareBin"] = pd.qcut(dataset.Fare, 4, labels=get_bin_labels("FareBin", 4))
    dataset["AgeBin"] = pd.cut(dataset.Age.astype(int), 5, labels=get_bin_labels("AgeBin", 5))

    # Remove now-unrelevant variables
    drop_column = ["Name", "SibSp", "Parch", "Fare", "Age"]
    dataset.drop(drop_column, axis=1, inplace=True)

Let's have a look on the titles' distribution:

In [10]:
data = pd.concat(datasets)
titles = data.Title.value_counts()
print(titles)

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Major         2
Ms            2
Lady          1
Sir           1
Mme           1
Don           1
Capt          1
Countess      1
Jonkheer      1
Dona          1
Name: Title, dtype: int64


We can clearly observe that only a few titles represent most of the individuals. In order to prevent the model from becoming overly specific, we decide to group all the "uncommon" titles together:

In [11]:
uncommon_titles = titles[titles < 10].index

for dataset in datasets:
    dataset.Title.replace(uncommon_titles, "Rare", inplace=True)

### Dummification

Finally, we can "dummify" the remaining categorical variables. Dummification is a common technique of transforming categorical (non-numerical) data into numerical data without having to map values or consider any order between each of them. The idea is to take all the different values found in a variable and create a new binary variable associated to each one of them. 

For example, the "Embarked" variable has three categorical values: "S", "C" and "Q". Dummifying the data will create three new variables, "Embarked_S", "Embarked_C" and "Embarked_Q", and set the value of "Embarked_S" (resp. "Embarked_C" and "Embarked_Q") to 1 for each data point initially labeled with "S" (resp. "C" and "Q") in the "Embarked" variable, else 0.

We also extract the input and target values used for training.

In [12]:
target = "Survived"
categorical_features = train_data.select_dtypes(exclude=np.number).columns

x_train = pd.get_dummies(train_data, prefix=categorical_features)
x_train = x_train.drop(columns=[target])

x_test = pd.get_dummies(test_data, prefix=categorical_features)

y_train = train_data[target]

Computing the predictions in FHE on a test set of about 400 data points can take up to several hours. We therefore reduce the test size in order to be able to run the notebook much faster. However, it is necessary to remove the following line for proper submission.

In [13]:
# Remove this line for proper submission
x_test = x_test.head(5)

## Training 
### Training with XGBoost

Let's train a model using XGBoost. Since several parameters have to be fixed beforehand, we use scikit-learn's GridSearchCV method to perform cross validation in order to maximise our chance to find the best ones. 

In [14]:
# Instantiate the Cross-Validation generator
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

# Set the parameters to tune
param_grid = {
    "max_depth": [5, 10, 15],
    "n_estimators": [5, 10, 100],
    "learning_rate": [0.01, 0.1, 1],
}

# Instantiate and fit the model
model = GridSearchCV(XGBClassifier(), param_grid, cv=cv, scoring="roc_auc")
model.fit(x_train, y_train);

### Training with Concrete-ML

Now, let's do the same with Concrete-ML's XGBClassifier method. 

In order to do so, we need to specify the number of bits over which inputs, outputs and weights will be quantized. This value can influence the precision of the model as well as its inference running time. In our case, setting this value to 2 bits outputs a satisfying accuracy score while running faster. 

In [15]:
# The Concrete-ML model needs an additional parameter used for quantization
param_grid["n_bits"] = [2]

x_train = x_train.astype(np.float32)

# Instantiate and fit the model
concrete_model = GridSearchCV(ConcreteXGBClassifier(), param_grid, cv=cv, scoring="roc_auc")
concrete_model.fit(x_train, y_train);

## Predicting the Outcomes

In [16]:
# Computing the predictions in clear using XGBoost
clear_predictions = model.predict(x_test)

# Computing the predictions in clear using Concrete-ML
clear_quantized_predictions = concrete_model.predict(x_test)

# Compiling the Concrete-ML model on a subset
fhe_circuit = concrete_model.best_estimator_.compile(x_train.head(100).to_numpy())

# Generating the keys
# This step is not necessary, keygen() is called directly within the predict method. However, since
# the keys are stored in cache by default, it is useful to run it beforehand in order to be able to
# measure the prediction executing time separately from the key generation one
time_begin = time.time()
fhe_circuit.keygen()
key_generation_duration = time.time() - time_begin

# Computing the predictions in FHE using Concrete-ML
time_begin = time.time()
fhe_predictions = concrete_model.best_estimator_.predict(x_test, execute_in_fhe=True)
prediction_duration = time.time() - time_begin

### Compare the predictions

Let's see how different the predictions are between the XGBoost model and the Concrete-ML ones:

In [17]:
nb_equal_preds = np.sum(clear_quantized_predictions == clear_predictions)
clear_prediction_similarity = nb_equal_preds / len(clear_predictions) * 100
print(
    "Prediction similarity between Concrete-ML (clear) and XGBoost (clear) models: "
    f"{clear_prediction_similarity:.2f}%",
)

Prediction similarity between Concrete-ML (clear) and XGBoost (clear) models: 100.00%


In [18]:
nb_equal_preds = np.sum(fhe_predictions == clear_predictions)
fhe_prediction_similarity = nb_equal_preds / len(clear_predictions) * 100
print(
    "Prediction similarity between Concrete-ML (FHE) and XGBoost (clear) models: "
    f"{fhe_prediction_similarity:.2f}%",
)

print(f"Key generation time: {key_generation_duration:.04f}s")
print(f"Execution time per inference: {prediction_duration / len(clear_predictions):.04f} s")

Prediction similarity between Concrete-ML (FHE) and XGBoost (clear) models: 100.00%
Key generation time: 22.5014s
Execution time per inference: 396.0776 s


### Save the Kaggle Submission File

When submitted to the Kaggle platform, this FHE model outputs an accuracy of around 77%.

In fact, using the given dataset, most of the submitted notebooks do not seem to exceed 79% of accuracy. Therefore, additional pre-processing and feature engineering might help increasing our current score but probably not by much.

In [19]:
try:
    submission = pd.DataFrame(
        {
            "PassengerId": test_ids,
            "Survived": fhe_predictions,
        }
    )
    submission.to_csv("titanic_submission.csv", index=False)
except ValueError:
    print("The predictions should not be saved if the test data was reduced for faster runs.")

The predictions should not be saved if the test data was reduced for faster runs.
